# Predictive Modeling (Detailed)
In this notebook, we:
- Try different models on the [detailed features](./eda_detailed.ipynb) and assess their performance.

In [1]:
from helper import *

# load in features compact
fdet = pd.read_csv('data/cleaned/features_detailed.csv')

# sort cols
fdet = fdet.reindex(sorted(fdet.columns), axis=1)

# view
print(fdet.shape)
fdet.sample()

(2204, 90)


,FG2_pct_x,FG2_pct_y,FG3_pct_x,FG3_pct_y,FT_pct_x,FT_pct_y,NumOT,PlayIn_x,PlayIn_y,Score_x,Score_y,Season,Seed_num_x,Seed_num_y,TeamID_x,TeamID_y,close_games_pct_x,close_games_pct_y,close_games_win_pct_x,close_games_win_pct_y,mean_FG2A_x,mean_FG2A_y,mean_FG3A_x,mean_FG3A_y,mean_FTA_x,mean_FTA_y,mean_TO_against_x,mean_TO_against_y,mean_TO_x,mean_TO_y,mean_ast_against_x,mean_ast_against_y,mean_ast_x,mean_ast_y,mean_blk_against_x,mean_blk_against_y,mean_blk_x,mean_blk_y,mean_fouls_against_x,mean_fouls_against_y,mean_fouls_x,mean_fouls_y,mean_pts_against_x,mean_pts_against_y,mean_pts_x,mean_pts_y,mean_reb_against_x,mean_reb_against_y,mean_reb_x,mean_reb_y,mean_score_diff_x,mean_score_diff_y,mean_stl_against_x,mean_stl_against_y,mean_stl_x,mean_stl_y,num_games_x,num_games_y,std_TO_against_x,std_TO_against_y,std_TO_x,std_TO_y,std_ast_against_x,std_ast_against_y,std_ast_x,std_ast_y,std_blk_against_x,std_blk_against_y,std_blk_x,std_blk_y,std_fouls_against_x,std_fouls_against_y,std_fouls_x,std_fouls_y,std_pts_against_x,std_pts_against_y,std_pts_x,std_pts_y,std_reb_against_x,std_reb_against_y,std_reb_x,std_reb_y,std_score_diff_x,std_score_diff_y,std_stl_against_x,std_stl_against_y,std_stl_x,std_stl_y,win_pct_x,win_pct_y
1265,0.517343,0.55913,0.348509,0.36236,0.696801,0.715613,0.0,0.0,0.0,82.0,65.0,2024.0,2.0,15.0,1235.0,1355.0,0.058824,0.258065,0.5,0.5,39.852941,34.096774,18.735294,22.967742,21.147059,17.354839,16.764706,10.709677,10.205882,10.935484,12.117647,11.870968,15.735294,12.709677,3.470588,2.967742,3.058824,3.0,18.441176,17.354839,16.470588,14.83871,61.294118,71.645161,75.558824,75.516129,29.794118,31.290323,31.0,31.483871,14.264706,3.870968,6.147059,6.709677,10.441176,5.935484,34.0,31.0,4.911698,3.475568,4.318946,3.054346,3.539943,3.730433,4.876094,3.100468,1.98838,2.105293,1.739752,1.770122,3.893983,3.911934,3.543969,3.916604,12.013955,12.210783,13.089879,8.559093,6.104044,4.845572,5.331439,5.993724,18.475715,12.711522,3.211083,2.355045,3.783466,2.24997,0.794118,0.612903


# Flip Values

In [2]:
# ensure team_x always wins
fdet.query('Score_x > Score_y').shape

(2204, 90)

Currently, the winner is always on the left side of the row (team 'x'). To prevent models from learning this positional encoding, we will duplicate rows similar to how we duplicated the regular season data. This will also double our pool of training examples.

In [3]:
# flip x and y col values
flipped = fdet.copy()
for col in flipped.columns:
    if col[-2:] == '_x':
        col_y = col.replace('_x', '_y')
        flipped.loc[:, col], flipped.loc[:, col_y] = flipped.loc[:, col_y].copy(), flipped.loc[:, col].copy()

# rearrange cols
flipped = flipped.reindex(sorted(flipped.columns), axis=1)

# check
display(fdet.head(1))
display(flipped.head(1))

,FG2_pct_x,FG2_pct_y,FG3_pct_x,FG3_pct_y,FT_pct_x,FT_pct_y,NumOT,PlayIn_x,PlayIn_y,Score_x,Score_y,Season,Seed_num_x,Seed_num_y,TeamID_x,TeamID_y,close_games_pct_x,close_games_pct_y,close_games_win_pct_x,close_games_win_pct_y,mean_FG2A_x,mean_FG2A_y,mean_FG3A_x,mean_FG3A_y,mean_FTA_x,mean_FTA_y,mean_TO_against_x,mean_TO_against_y,mean_TO_x,mean_TO_y,mean_ast_against_x,mean_ast_against_y,mean_ast_x,mean_ast_y,mean_blk_against_x,mean_blk_against_y,mean_blk_x,mean_blk_y,mean_fouls_against_x,mean_fouls_against_y,mean_fouls_x,mean_fouls_y,mean_pts_against_x,mean_pts_against_y,mean_pts_x,mean_pts_y,mean_reb_against_x,mean_reb_against_y,mean_reb_x,mean_reb_y,mean_score_diff_x,mean_score_diff_y,mean_stl_against_x,mean_stl_against_y,mean_stl_x,mean_stl_y,num_games_x,num_games_y,std_TO_against_x,std_TO_against_y,std_TO_x,std_TO_y,std_ast_against_x,std_ast_against_y,std_ast_x,std_ast_y,std_blk_against_x,std_blk_against_y,std_blk_x,std_blk_y,std_fouls_against_x,std_fouls_against_y,std_fouls_x,std_fouls_y,std_pts_against_x,std_pts_against_y,std_pts_x,std_pts_y,std_reb_against_x,std_reb_against_y,std_reb_x,std_reb_y,std_score_diff_x,std_score_diff_y,std_stl_against_x,std_stl_against_y,std_stl_x,std_stl_y,win_pct_x,win_pct_y
0,0.510172,0.484202,0.350534,0.340757,0.701429,0.657848,0.0,0.0,0.0,80.0,51.0,2003.0,1.0,16.0,1112.0,1436.0,0.178571,0.241379,0.6,0.428571,45.642857,40.37931,20.071429,15.482759,25.0,19.551724,16.857143,13.0,14.785714,14.068966,15.464286,13.275862,17.642857,14.206897,2.392857,3.655172,4.214286,2.965517,22.071429,17.931034,17.75,15.896552,70.25,63.137931,85.214286,67.793103,36.392857,31.448276,42.821429,38.689655,14.964286,4.655172,5.964286,7.103448,8.464286,6.862069,28.0,29.0,5.147558,4.234214,4.458509,3.483953,3.156207,4.096135,4.339434,4.857831,1.663091,3.003282,2.006603,1.721352,4.438289,4.333681,2.989178,4.04744,9.359586,10.384481,10.379314,11.995176,5.349554,7.15435,4.546497,5.352592,12.556486,12.601802,3.06089,2.968223,3.646227,3.502286,0.892857,0.655172


,FG2_pct_x,FG2_pct_y,FG3_pct_x,FG3_pct_y,FT_pct_x,FT_pct_y,NumOT,PlayIn_x,PlayIn_y,Score_x,Score_y,Season,Seed_num_x,Seed_num_y,TeamID_x,TeamID_y,close_games_pct_x,close_games_pct_y,close_games_win_pct_x,close_games_win_pct_y,mean_FG2A_x,mean_FG2A_y,mean_FG3A_x,mean_FG3A_y,mean_FTA_x,mean_FTA_y,mean_TO_against_x,mean_TO_against_y,mean_TO_x,mean_TO_y,mean_ast_against_x,mean_ast_against_y,mean_ast_x,mean_ast_y,mean_blk_against_x,mean_blk_against_y,mean_blk_x,mean_blk_y,mean_fouls_against_x,mean_fouls_against_y,mean_fouls_x,mean_fouls_y,mean_pts_against_x,mean_pts_against_y,mean_pts_x,mean_pts_y,mean_reb_against_x,mean_reb_against_y,mean_reb_x,mean_reb_y,mean_score_diff_x,mean_score_diff_y,mean_stl_against_x,mean_stl_against_y,mean_stl_x,mean_stl_y,num_games_x,num_games_y,std_TO_against_x,std_TO_against_y,std_TO_x,std_TO_y,std_ast_against_x,std_ast_against_y,std_ast_x,std_ast_y,std_blk_against_x,std_blk_against_y,std_blk_x,std_blk_y,std_fouls_against_x,std_fouls_against_y,std_fouls_x,std_fouls_y,std_pts_against_x,std_pts_against_y,std_pts_x,std_pts_y,std_reb_against_x,std_reb_against_y,std_reb_x,std_reb_y,std_score_diff_x,std_score_diff_y,std_stl_against_x,std_stl_against_y,std_stl_x,std_stl_y,win_pct_x,win_pct_y
0,0.484202,0.510172,0.340757,0.350534,0.657848,0.701429,0.0,0.0,0.0,51.0,80.0,2003.0,16.0,1.0,1436.0,1112.0,0.241379,0.178571,0.428571,0.6,40.37931,45.642857,15.482759,20.071429,19.551724,25.0,13.0,16.857143,14.068966,14.785714,13.275862,15.464286,14.206897,17.642857,3.655172,2.392857,2.965517,4.214286,17.931034,22.071429,15.896552,17.75,63.137931,70.25,67.793103,85.214286,31.448276,36.392857,38.689655,42.821429,4.655172,14.964286,7.103448,5.964286,6.862069,8.464286,29.0,28.0,4.234214,5.147558,3.483953,4.458509,4.096135,3.156207,4.857831,4.339434,3.003282,1.663091,1.721352,2.006603,4.333681,4.438289,4.04744,2.989178,10.384481,9.359586,11.995176,10.379314,7.15435,5.349554,5.352592,4.546497,12.601802,12.556486,2.968223,3.06089,3.502286,3.646227,0.655172,0.892857


Now that we have a flipped version of our features, we can combine the 2 dataframes.

In [4]:
# combine
fdet = pd.concat([fdet, flipped], axis=0)

# check
print(fdet.shape)
display(fdet.sample())

(4408, 90)


,FG2_pct_x,FG2_pct_y,FG3_pct_x,FG3_pct_y,FT_pct_x,FT_pct_y,NumOT,PlayIn_x,PlayIn_y,Score_x,Score_y,Season,Seed_num_x,Seed_num_y,TeamID_x,TeamID_y,close_games_pct_x,close_games_pct_y,close_games_win_pct_x,close_games_win_pct_y,mean_FG2A_x,mean_FG2A_y,mean_FG3A_x,mean_FG3A_y,mean_FTA_x,mean_FTA_y,mean_TO_against_x,mean_TO_against_y,mean_TO_x,mean_TO_y,mean_ast_against_x,mean_ast_against_y,mean_ast_x,mean_ast_y,mean_blk_against_x,mean_blk_against_y,mean_blk_x,mean_blk_y,mean_fouls_against_x,mean_fouls_against_y,mean_fouls_x,mean_fouls_y,mean_pts_against_x,mean_pts_against_y,mean_pts_x,mean_pts_y,mean_reb_against_x,mean_reb_against_y,mean_reb_x,mean_reb_y,mean_score_diff_x,mean_score_diff_y,mean_stl_against_x,mean_stl_against_y,mean_stl_x,mean_stl_y,num_games_x,num_games_y,std_TO_against_x,std_TO_against_y,std_TO_x,std_TO_y,std_ast_against_x,std_ast_against_y,std_ast_x,std_ast_y,std_blk_against_x,std_blk_against_y,std_blk_x,std_blk_y,std_fouls_against_x,std_fouls_against_y,std_fouls_x,std_fouls_y,std_pts_against_x,std_pts_against_y,std_pts_x,std_pts_y,std_reb_against_x,std_reb_against_y,std_reb_x,std_reb_y,std_score_diff_x,std_score_diff_y,std_stl_against_x,std_stl_against_y,std_stl_x,std_stl_y,win_pct_x,win_pct_y
1002,0.5434,0.545982,0.363095,0.351275,0.657046,0.684515,0.0,0.0,0.0,58.0,54.0,2018.0,3.0,9.0,1276.0,1199.0,0.147059,0.16129,0.6,0.8,32.529412,38.935484,24.705882,22.774194,17.323529,22.290323,12.852941,14.096774,9.176471,13.0,10.470588,12.83871,14.588235,15.258065,3.235294,2.967742,3.088235,5.16129,17.705882,18.806452,15.705882,18.290323,63.529412,74.483871,73.647059,81.774194,32.617647,34.935484,33.323529,38.032258,10.117647,7.290323,4.088235,5.870968,6.176471,6.709677,34.0,31.0,4.806135,4.901393,2.3928,3.172801,4.69839,4.359639,3.985714,4.732637,1.985689,2.575317,1.928536,2.59611,3.888829,4.784833,4.034168,4.345063,11.133448,12.842302,10.548251,13.300901,5.039328,5.887475,5.34688,6.954537,14.446903,15.423345,1.747165,2.093513,3.069603,2.946111,0.794118,0.645161


# Create Labels

In [5]:
# create regression label, drop cols
fdet['score_diff_x'] = fdet['Score_x'] - fdet['Score_y']

# create adjusted score diff col (score diff is inversely scaled by NumOT periods)
fdet['score_diff_adj_x'] = fdet['score_diff_x'] / (2 ** fdet['NumOT'])

# create binary label
fdet['win_x'] = fdet['score_diff_x'].apply(lambda x: 1 if x > 0 else 0)

# create upset win label (Seed_num_x > Seed_num_y & win_x == 1) | (Seed_num_x < Seed_num_y & win_x == 0)
# fdet['win_upset'] = ((fdet['Seed_num_x'] > fdet['Seed_num_y']) & (fdet['win_x'] == 1)) | ((fdet['Seed_num_x'] < fdet['Seed_num_y']) & (fdet['win_x'] == 0))

# check
fdet.sample()[['Score_x', 'Score_y', 'score_diff_x', 'win_x']]

,Score_x,Score_y,score_diff_x,win_x
783,64.0,76.0,-12.0,0


In [28]:
# proportion of upset wins for men and women
# fdet.query('TeamID_x < 3000')['win_upset'].value_counts(normalize=True), fdet.query('TeamID_x > 3000')['win_upset'].value_counts(normalize=True)

- Men's upsets since 2003: __29%__
- Women's upsets since 2010: __21%__

# Chalk Bracket
Here, we will simply predict the better seed to win each game. If seeds are equal (in rounds 5 and 6), we will predict the team with the better regular season winning percentage.

In [29]:
# split genders for winning rows
fdet_mens, fdet_womens = split_genders(fdet[fdet['win_x'] == 1], id_col='TeamID_x')

# get dummy preds for men and calculate accuracy
mchalk_preds = get_dummy_preds(fdet_mens)
mchalk_acc = accuracy_score(fdet_mens['win_x'], mchalk_preds)

# women
wchalk_preds = get_dummy_preds(fdet_womens)
wchalk_acc = accuracy_score(fdet_womens['win_x'], wchalk_preds)

# show
print(f"Accuracy of dummy predictions in men's tournaments (39 brackets): {mchalk_acc*100:.2f}%.")
print(f"Accuracy of dummy predictions in women's tournaments (26 brackets): {wchalk_acc*100:.2f}%")

Accuracy of dummy predictions in men's tournaments (39 brackets): 70.57%.
Accuracy of dummy predictions in women's tournaments (26 brackets): 78.46%


- We can see that the better seed wins about 8% more often in women's tournaments (post 2000). This is slightly more often than the chalk bracket in the full compact data.

# Finalize Columns
We will one-hot encode categorical columns, then drop non-feature columns. We will also create seperate feature subsets that only contain general scoring and win data.

In [6]:
# get copy
X = fdet.copy()

# non-feature columns
dropped_cols = ['Score_x', 'Score_y', 'score_diff_x', 'NumOT']

# split genders
X_mens, X_womens = split_genders(X, id_col='TeamID_x')

# drop non-feature columns
X_mens = X_mens.drop(columns=dropped_cols + ['Season', 'TeamID_x', 'TeamID_y'])
X_womens = X_womens.drop(columns=dropped_cols + ['Season', 'TeamID_x', 'TeamID_y'])

# number of features
X_mens.shape, X_womens.shape

((2644, 86), (1764, 86))

- We have roughly 2000 rows of training data for both men and women.
- 3 of these columns are labels. Thus, we have 83 features.
- Each gender will be trained seperately, as we have already found multiple statistically-significant differences in the data.

# Test Models
We will experiment with 5 regression models, 5 classification models, and 3 scalers. We will also try each of the 2 different feature subsets.

In [ ]:
# define models and scalers
models_class = [LogisticRegression(n_jobs=-1), DecisionTreeClassifier(max_depth=10), RandomForestClassifier(n_jobs=-1, max_depth=10), XGBClassifier(n_jobs=-1, max_depth=5), SVC()]

# create a df to hold model performance
models_df = pd.DataFrame(columns=['Gender', 'Target', 'Model', 'Model_Params', 'Scaler', 'Num_Features', 'Train_R2', 'Val_R2', 'Train_RMSE', 'Val_RMSE', 'Train_LogLoss', 'Val_LogLoss', 'Train_Acc', 'Val_Acc']).astype({
    'Gender': 'object', 'Target': 'object', 'Model': 'object', 'Model_Params': 'object', 'Scaler': 'object', 'Num_Features': 'int', 'Train_R2': 'float', 'Val_R2': 'float', 'Train_RMSE': 'float', 'Val_RMSE': 'float', 
    'Train_LogLoss': 'float', 'Val_LogLoss': 'float', 'Train_Acc': 'float', 'Val_Acc': 'float'})
models_df

,Gender,Target,Model,Model_Params,Scaler,Num_Features,Train_R2,Val_R2,Train_RMSE,Val_RMSE,Train_LogLoss,Val_LogLoss,Train_Acc,Val_Acc


## Men's
We're trying to beat the chalk accuracy of __70.57%__.

In [13]:
# run models
for model in tqdm(models_class, desc="Running Models"):
    cross_val_model(estimator=model, df=X_mens, target_col='win_x', gender='M', scaler=RobustScaler(), models_df=models_df)

Running Models: 100%|██████████| 5/5 [00:26<00:00,  5.24s/it]


In [14]:
# inspect
models_df.query("Gender == 'M'").sort_values(by='Val_LogLoss').head()

,Gender,Target,Model,Model_Params,Scaler,Num_Features,Train_R2,Val_R2,Train_RMSE,Val_RMSE,Train_LogLoss,Val_LogLoss,Train_Acc,Val_Acc
0,M,win_x,LogisticRegression,"{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': -1, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}",RobustScaler,84,0.0,0.0,0.0,0.0,1.009239e+01,10.932829,0.719995,0.696678
1,M,win_x,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': -1, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}",RobustScaler,84,0.0,0.0,0.0,0.0,3.801866e-01,11.287186,0.989452,0.686847
2,M,win_x,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}",RobustScaler,84,0.0,0.0,0.0,0.0,4.648603e+00,11.600791,0.871029,0.678146
3,M,win_x,XGBClassifier,"{'objective': 'binary:logistic', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': None, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 5, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': None, 'n_jobs': -1, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': None, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}",RobustScaler,84,0.0,0.0,0.0,0.0,2.220446e-16,12.119189,1.000000,0.663764
4,M,win_x,DecisionTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'random_state': None, 'splitter': 'best'}",RobustScaler,84,0.0,0.0,0.0,0.0,2.623402e+00,13.482264,0.927216,0.625946


- Best performance was __69.67%__ accuracy using a __Logistic Regression__, but this is worse than our results on the compact data.

## Women's
We're trying to beat the chalk accuracy of __78.46%__.

In [15]:
# run models
for model in tqdm(models_class, desc="Running Models"):
    cross_val_model(estimator=model, df=X_womens, target_col='win_x', gender='W', scaler=RobustScaler(), models_df=models_df)

Running Models: 100%|██████████| 5/5 [00:14<00:00,  2.96s/it]


In [16]:
# inspect
models_df.query("Gender == 'W'").sort_values(by='Val_LogLoss').head()

,Gender,Target,Model,Model_Params,Scaler,Num_Features,Train_R2,Val_R2,Train_RMSE,Val_RMSE,Train_LogLoss,Val_LogLoss,Train_Acc,Val_Acc
5,W,win_x,LogisticRegression,"{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': -1, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}",RobustScaler,84,0.0,0.0,0.0,0.0,6.754188e+00,7.723491,0.812611,0.785718
7,W,win_x,SVC,"{'C': 1.0, 'break_ties': False, 'cache_size': 200, 'class_weight': None, 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': -1, 'probability': False, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}",RobustScaler,84,0.0,0.0,0.0,0.0,3.795963e+00,8.132268,0.894684,0.774377
6,W,win_x,RandomForestClassifier,"{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'n_estimators': 100, 'n_jobs': -1, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}",RobustScaler,84,0.0,0.0,0.0,0.0,7.945417e-02,8.398616,0.997796,0.766988
8,W,win_x,XGBClassifier,"{'objective': 'binary:logistic', 'base_score': None, 'booster': None, 'callbacks': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': None, 'device': None, 'early_stopping_rounds': None, 'enable_categorical': False, 'eval_metric': None, 'feature_types': None, 'gamma': None, 'grow_policy': None, 'importance_type': None, 'interaction_constraints': None, 'learning_rate': None, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 5, 'max_leaves': None, 'min_child_weight': None, 'missing': nan, 'monotone_constraints': None, 'multi_strategy': None, 'n_estimators': None, 'n_jobs': -1, 'num_parallel_tree': None, 'random_state': None, 'reg_alpha': None, 'reg_lambda': None, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': None, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}",RobustScaler,84,0.0,0.0,0.0,0.0,2.220446e-16,8.683938,1.000000,0.759072
9,W,win_x,DecisionTreeClassifier,"{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'monotonic_cst': None, 'random_state': None, 'splitter': 'best'}",RobustScaler,84,0.0,0.0,0.0,0.0,1.023862e+00,10.421327,0.971594,0.710869


- Best performance was __78.57%__ accuracy using a __Logistic Regression__, but this is worse than our results on the compact data.

In [18]:
# save models_df
models_df.to_csv('models/models_detailed.csv', index=False)

# Hyperparameter Tuning
Here we will try out a PyTorch neural network with a two-tower approach, first training a dense NN on the 42 team_x features, and another NN on the 42 team_y features, then, we will pass the two outputs into a final NN that outputs the probability that team_x wins. Maybe this model will be strong enough to compete with our 2 best models from the [compact predictions](./preds_compact.ipynb).

In [30]:
# run bayes opt nn (mens)
bayes_opt_nn(df=X_mens, init_points=10, n_iter=100)

Using device: cuda
|   iter    |  target   | combin... | dropou... | hidden... | hidden... |    lr     | weight... |
-------------------------------------------------------------------------------------------------
| 1         | -0.6932   | 4.635     | 0.5015    | 127.9     | 35.99     | 0.01422   | 0.02186   |
| 2         | -0.6932   | 108.3     | 0.2985    | 23.35     | 89.76     | 0.01669   | 0.08786   |
| 3         | -0.6932   | 243.5     | 0.131     | 179.6     | 147.5     | 0.0898    | 0.06669   |
| 4         | -0.6932   | 141.2     | 0.6619    | 100.2     | 178.4     | 0.08249   | 0.04657   |
| 5         | -0.6932   | 240.9     | 0.743     | 251.8     | 45.09     | 0.05211   | 0.09455   |
| 6         | -0.6932   | 167.0     | 0.7887    | 248.6     | 49.13     | 0.02432   | 0.08734   |
| 7         | -0.6932   | 232.2     | 0.7066    | 90.11     | 119.7     | 0.06106   | 0.04963   |
| 8         | -0.6932   | 123.4     | 0.4484    | 223.1     | 49.59     | 0.06877   | 0.07036   |
|

- This accuracy is still worse than our results using the compact data.

In [ ]:
# run bayes opt nn (womens)
bayes_opt_nn(df=X_womens, init_points=10, n_iter=100)

Using device: cuda
|   iter    |  target   | combin... | dropou... | hidden... | hidden... |    lr     | weight... |
-------------------------------------------------------------------------------------------------
| 1         | -0.4951   | 4.635     | 0.5015    | 127.9     | 35.99     | 0.01422   | 0.02186   |
| 2         | -0.6934   | 108.3     | 0.2985    | 23.35     | 89.76     | 0.01669   | 0.08786   |
| 3         | -0.6934   | 243.5     | 0.131     | 179.6     | 147.5     | 0.0898    | 0.06669   |
| 4         | -0.6934   | 141.2     | 0.6619    | 100.2     | 178.4     | 0.08249   | 0.04657   |
| 5         | -0.6934   | 240.9     | 0.743     | 251.8     | 45.09     | 0.05211   | 0.09455   |
| 6         | -0.6934   | 167.0     | 0.7887    | 248.6     | 49.13     | 0.02432   | 0.08734   |
| 7         | -0.6934   | 232.2     | 0.7066    | 90.11     | 119.7     | 0.06106   | 0.04963   |
| 8         | -0.6934   | 123.4     | 0.4484    | 223.1     | 49.59     | 0.06877   | 0.07036   |
|

- This accuracy is still worse than our results using the compact data.